In [6]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,  TfidfTransformer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_iris
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy.sparse import csr_matrix
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
import logging
import json
from sklearn.svm import SVC
import embeddingvectorizer
from sklearn.ensemble import ExtraTreesClassifier
import gensim
import os


path_to_embeddings='/home/anne/tmpanne/fullsample'
path_to_data='/home/anne/tmpanne/AEM_data/'
dataset = 'dataset_vermeer.pkl'
outputpath = '/home/anne/tmpanne/AEM_output/'

#word_embedding_path = '/home/anne/tmpanne/fullsample/'

class classifier_analyzer():
    
    def __init__(self, path_to_data, path_to_embeddings, dataset):
        self.nmodel = 0
        df = pd.read_pickle(path_to_data + dataset)
        logging.info("... loading the data...\n\nthis is length of the dataframe: {}".format(len(df)))
        self.test_size = 0.2
        self.data = df['text']
        self.labels = df['topic']
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.data, self.labels, test_size=self.test_size, random_state=42)
        self.basepath = path_to_embeddings
        self.names = ["Passive Agressive", "SGDClassifier" , "SVM", "ET"]
        self.classifiers = [PassiveAggressiveClassifier(), 
                            SGDClassifier(),
                            SVC(),
                            ExtraTreesClassifier() ]
        self.parameters = [

                    {'clf__loss': ('hinge', 'squared_hinge'),
                    'clf__C': (0.01, 0.5, 1.0)   ,
                    'clf__fit_intercept': (True, False) ,
                    #'vect__ngram_range': [(1, 1), (1, 2)] ,
                #    'tfidf__use_idf' :(True ,False),
                    'clf__max_iter': (5 ,10 ,15) } ,

                    {'clf__max_iter': (20, 30) ,
                    'clf__alpha': (1e-2, 1e-3, 1e-5),
                    'clf__penalty': ('l2', 'elasticnet')} ,

                    {'clf__C': [1, 10, 100, 1000],
                    'clf__gamma': [0.001, 0.0001],
                    'clf__kernel': ['rbf', 'linear']},


                    { "clf__max_features": ['auto', 'sqrt', 'log2'] }

                     ]


    def get_w2v_model(self):
            '''yields a dict with one item. key is the filename, value the gensim model'''

            filenames = [e for e in os.listdir(self.basepath) if not e.startswith('.')]

            for fname in filenames:
                model = {}
                path = os.path.join(self.basepath, fname)
                print("\nLoading gensim model")

                if fname.startswith('w2v'):
                    mod = gensim.models.Word2Vec.load(path)
                else:
                    mod = gensim.models.KeyedVectors.load_word2vec_format(path)

                model['gensimmodel'] = dict(zip(mod.wv.index2word, mod.wv.vectors))
                model['filename'] = fname
                self.nmodel +=1
                print("loaded gensim model nr {}, named: {}".format(self.nmodel, model['filename']))
                yield model


    def get_vectorizer(self, vectorizer, model):
        logging.info("the vectorizer is: {}".format(vectorizer))
        
       # if self.vectorizer == 'tfidf':
       #     VECT = TfidfVectorizer()

      #  elif self.vectorizer == 'count':
      #      VECT = CountVectorizer
            
           # logging.info('... Loading W2V model....:{}'.format(self.model['filename']))
        vec = {}   
        vec['filename'] = vectorizer
        if vectorizer == 'w2v_count':
            s = embeddingvectorizer.EmbeddingCountVectorizer(model['gensimmodel'], 'mean')
        elif vectorizer == 'w2v_tfidf':
            s = embeddingvectorizer.EmbeddingTfidfVectorizer(model['gensimmodel'], 'mean')
        vec['vectorizer'] = s
        print("loaded vectorizer, named: {}".format(vec['filename']))

        yield vec


    def gridsearch_with_classifiers_embeddings(self):
        class_report = []
        results = []
        
        for model in self.get_w2v_model():
            for v in ["w2v_count", "w2v_tfidf"]:
                for vec in self.get_vectorizer(v, model):
                    print("loaded the vectorizer: {}".format(vec['filename'])) 
                    for name, classifier, params in zip(self.names, self.classifiers, self.parameters):
                        my_dict = {}
                        
                        logging.info("Starting gridsearch CV..")
                        logging.info("Classifier name: {}\n classifier:{}\n params{}\n".format(name, classifier, params)) 

                        clf_pipe = Pipeline([ ('vect', vec['vectorizer']), ('clf', classifier), ])

                        gs_clf = GridSearchCV(clf_pipe, param_grid=params, cv=2)
                        clf = gs_clf.fit(self.X_train, self.y_train)
                        score = clf.score(self.X_test, self.y_test)

                        logging.info("{} score: {}".format(name, score))
                        logging.info("{} are the best estimators".format(clf.best_estimator_))

                        results_to_dict = classification_report((clf.best_estimator_.predict(self.X_test)), self.y_test, output_dict= True)

                        results_to_dict['classifier'] = name
                        results_to_dict['parameters'] = clf.best_params_
                        results_to_dict['vectorizer'] = vec['filename']
                        results_to_dict['model'] = model['filename']

                        logging.info("Created dictionary with classification report: \n\n{}".format(results_to_dict))
                        class_report.append(results_to_dict)
                        
                        y_hats = clf.predict(self.X_test)
                        results.append({"predicted": y_hats,
                                        "actual" : self.y_test.values  ,
                                        "classifier": name} )
                        
        return class_report, results

    def gridsearch_with_classifiers_baseline(self):
        
        for vec in [CountVectorizer(), TfidfVectorizer()]:
            print("loaded the vectorizer: {}".format(vec)) 
            for name, classifier, params in zip(self.names, self.classifiers, self.parameters):
                my_dict = {}

                logging.info("Starting gridsearch CV..")
                logging.info("Classifier name: {}\n classifier:{}\n params{}\n".format(name, classifier, params)) 

                clf_pipe = Pipeline([ ('vect', vec), ('clf', classifier), ])

                gs_clf = GridSearchCV(clf_pipe, param_grid=params, cv=2)
                clf = gs_clf.fit(self.X_train, self.y_train)
                score = clf.score(self.X_test, self.y_test)

                logging.info("{} score: {}".format(name, score))
                logging.info("{} are the best estimators".format(clf.best_estimator_))

                results_to_dict = classification_report((clf.best_estimator_.predict(self.X_test)), self.y_test, output_dict= True)

                results_to_dict['classifier'] = name
                results_to_dict['parameters'] = clf.best_params_
                results_to_dict['vectorizer'] = vec
                
                logging.info("Created dictionary with classification report: \n\n{}".format(results_to_dict))
                class_report.append(results_to_dict)

                y_hats = clf.predict(self.X_test)
                results.append({"predicted": y_hats,
                                "actual" : self.y_test.values  ,
                                "classifier": name} )


def get_scores(path_to_data, path_to_embeddings, dataset, outputpath):
    a = classifier_analyzer(path_to_data=path_to_data, path_to_embeddings=path_to_embeddings, dataset=dataset)
    class_report, results = a.gridsearch_with_classifiers_baseline()

    fname_accuracy = '{}baseline_classreport.json'.format(outputpath)
    fname_true_predicted = '{}baseline_true_predicted.json'.format(outputpath)

    with open(fname_accuracy, mode = 'w') as fo:
        json.dump(class_report, fo)

    data = pd.DataFrame.from_dict(results)

    predicted = data.predicted.apply(pd.Series).merge(data, right_index = True, left_index = True) \
        .drop(["predicted"], axis = 1).melt(id_vars = ['classifier'], value_name = "Predicted label")

    actual = data.actual.apply(pd.Series).merge(data, right_index = True, left_index = True) \
        .drop(["predicted"], axis = 1).melt(id_vars = ['classifier'], value_name = "Actual label")

    df = pd.merge(predicted, actual, how = 'inner', left_index = True, right_index = True)
    
    df['Classifier'] = df['classifier_x']
    df = df[df.variable_x != 'actual']
    df = df[['Predicted label', 'Actual label', 'Classifier']]

    df.to_json(fname_true_predicted)

    
if __name__ == "__main__":

    logger = logging.getLogger()
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
    logging.root.setLevel(level=logging.INFO)
    
    get_scores(path_to_embeddings=path_to_embeddings,path_to_data=path_to_data, dataset=dataset, outputpath = outputpath)

#"w2v_count", "w2v_tfidf", "count", "tfidf"


2019-12-11 11:13:03,625 : INFO : ... loading the data...

this is length of the dataframe: 3486
2019-12-11 11:13:03,628 : INFO : Starting gridsearch CV..
2019-12-11 11:13:03,629 : INFO : Classifier name: Passive Agressive
 classifier:PassiveAggressiveClassifier(C=1.0, average=False, class_weight=None,
              early_stopping=False, fit_intercept=True, loss='hinge',
              max_iter=None, n_iter=None, n_iter_no_change=5, n_jobs=None,
              random_state=None, shuffle=True, tol=None,
              validation_fraction=0.1, verbose=0, warm_start=False)
 params{'clf__fit_intercept': (True, False), 'clf__C': (0.01, 0.5, 1.0), 'clf__loss': ('hinge', 'squared_hinge'), 'clf__max_iter': (5, 10, 15)}



loaded the vectorizer: CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)


2019-12-11 11:16:17,097 : INFO : Passive Agressive score: 0.7865329512893983
2019-12-11 11:16:17,099 : INFO : Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...=None, shuffle=True, tol=None,
              validation_fraction=0.1, verbose=0, warm_start=False))]) are the best estimators
2019-12-11 11:16:17,464 : INFO : Created dictionary with classification report: 

{'1': {'precision': 0.8048780487804879, 'support': 113, 'recall': 0.8761061946902655, 'f1-score': 0.8389830508474576}, 'macro avg': {'precision': 0.7152860090825017, 'support': 698, 'recall': 0.7440394701102254, 'f1-score': 0.7280183321563088}, '4': {'precision': 0.5526315789473685, 'support': 64, 'recall': 0.65625, 'f1-score': 0.6}, 'vectorizer': Co

NameError: name 'class_report' is not defined

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

for vec in [CountVectorizer(), TfidfVectorizer()]:
    print(vec)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)
TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)


In [ ]:
path_to_embeddings='/home/anne/tmpanne/AEM_small_sample/test'
path_to_data='/home/anne/tmpanne/AEM_data/'
#dataset = 'dataset_burscher.pkl'
dataset = 'dataset_vermeer.pkl'
outputpath = '/home/anne/tmpanne/AEM_output/'

a = classifier_analyzer(path_to_data=path_to_data, path_to_embeddings= path_to_embeddings, dataset = dataset)


In [ ]:
class_report, results = a.gridsearch_with_classifiers()


In [ ]:
def get_scores(path_to_data, path_to_embeddings, dataset, outputpath):
    a = classifier_analyzer(path_to_data=path_to_data, path_to_embeddings=path_to_embeddings, dataset=dataset)
    class_report, results = a.gridsearch_with_classifiers()
    
    fname_accuracy = '{}classreport.json'.format(outputpath)
    fname_predictions = '{}SML_predicted_actual_text_cleaned.json'.format(outputpath)

    with open(fname_accuracy, mode = 'w') as fo:
        json.dump(class_report, fo)

    data = pd.DataFrame.from_dict(results)

    predicted = data.predicted.apply(pd.Series).merge(data, right_index = True, left_index = True) \
        .drop(["predicted"], axis = 1).melt(id_vars = ['classifier'], value_name = "Predicted label")

    actual = data.actual.apply(pd.Series).merge(data, right_index = True, left_index = True) \
        .drop(["predicted"], axis = 1).melt(id_vars = ['classifier'], value_name = "Actual label")

    df = pd.merge(predicted, actual, how = 'inner', left_index = True, right_index = True)

    df['Classifier'] = df['classifier_x']
    df = df[df.variable_x != 'actual']
    df = df[['Predicted label', 'Actual label', 'Classifier']]

    df.to_json(fname_predictions)


In [ ]:
data = pd.DataFrame.from_dict(results)

predicted = data.predicted.apply(pd.Series) \
    .merge(data, right_index = True, left_index = True) \
    .drop(["predicted"], axis = 1) \
    .melt(id_vars = ['classifier'], value_name = "Predicted label")

actual = data.actual.apply(pd.Series) \
    .merge(data, right_index = True, left_index = True) \
    .drop(["predicted"], axis = 1) \
    .melt(id_vars = ['classifier'], value_name = "Actual label")

df = pd.merge(predicted, actual, how = 'inner', left_index = True, right_index = True)

df['Classifier'] = df['classifier_x']
df = df[df.variable_x != 'actual']
df = df[['Predicted label', 'Actual label', 'Classifier']]
df

In [ ]:
if __name__ == "__main__":

    logger = logging.getLogger()
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
    logging.root.setLevel(level=logging.INFO)

    get_scores(sample="newspaper_sample_only", vect = "count")
    get_scores(sample="pq_sample_only", vect = "count")
    get_scores(sample="RPA_sample", vect = "count")
    

In [ ]:
import argparse

parser = argparse.ArgumentParser(description='Parsing data...')
parser.add_argument('--word_embedding_path', type=str, required=True, help='Path of pretrained word embedding.')
parser.add_argument('--data_path', type=str, required=False, default='data/dataset_vermeer.pkl', help='Path of dataset with annotated data to be classified')
parser.add_argument('--output', type=str, required=False, default='tables_figures/', help='Path of output file (CSV formatted classification scores)')
args = parser.parse_args()

print('Arguments:')
print('word_embedding.path:', args.word_embedding_path)
print('output.path:', args.output)
print('data.path:', args.data_path)
print()

In [2]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,  TfidfTransformer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_iris
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy.sparse import csr_matrix
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
import logging
import json
from sklearn.svm import SVC
import embeddingvectorizer
from sklearn.ensemble import ExtraTreesClassifier
import gensim
import os


path_to_embeddings='/home/anne/tmpanne/fullsample'
path_to_data='/home/anne/tmpanne/AEM_data/'
dataset = 'dataset_vermeer.pkl'
outputpath = '/home/anne/tmpanne/AEM_output/'

#word_embedding_path = '/home/anne/tmpanne/fullsample/'

class classifier_analyzer():
    
    def __init__(self, path_to_data, path_to_embeddings, dataset):
        self.nmodel = 0
        df = pd.read_pickle(path_to_data + dataset)
        
        logging.info("... loading the data...\n\nthis is length of the dataframe: {}".format(len(df)))
        self.test_size = 0.2
        self.data = df['text']
        self.labels = df['topic']
        
        # validation data
        
        self.train, self.test = train_test_split(self.data, self.labels, test_size = 0.1)
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.data, self.labels, test_size=self.test_size, random_state=42)
        
        self.basepath = path_to_embeddings
        self.names = ["Passive Agressive", "SGDClassifier" , "SVM", "ET"]
        self.classifiers = [PassiveAggressiveClassifier(), 
                            SGDClassifier(),
                            SVC(),
                            ExtraTreesClassifier() ]
        self.parameters = [

                    {'clf__loss': ('hinge', 'squared_hinge'),
                    'clf__C': (0.01, 0.5, 1.0)   ,
                    'clf__fit_intercept': (True, False) ,
                    #'vect__ngram_range': [(1, 1), (1, 2)] ,
                #    'tfidf__use_idf' :(True ,False),
                    'clf__max_iter': (5 ,10 ,15) } ,

                    {'clf__max_iter': (20, 30) ,
                    'clf__alpha': (1e-2, 1e-3, 1e-5),
                    'clf__penalty': ('l2', 'elasticnet')} ,

                    {'clf__C': [1, 10, 100, 1000],
                    'clf__gamma': [0.001, 0.0001],
                    'clf__kernel': ['rbf', 'linear']},


                    { "clf__max_features": ['auto', 'sqrt', 'log2'] }

                     ]


    def get_w2v_model(self):
            '''yields a dict with one item. key is the filename, value the gensim model'''

            filenames = [e for e in os.listdir(self.basepath) if not e.startswith('.')]

            for fname in filenames:
                model = {}
                path = os.path.join(self.basepath, fname)
                print("\nLoading gensim model")

                if fname.startswith('w2v'):
                    mod = gensim.models.Word2Vec.load(path)
                else:
                    mod = gensim.models.KeyedVectors.load_word2vec_format(path)

                model['gensimmodel'] = dict(zip(mod.wv.index2word, mod.wv.vectors))
                model['filename'] = fname
                self.nmodel +=1
                print("loaded gensim model nr {}, named: {}".format(self.nmodel, model['filename']))
                yield model


    def get_vectorizer(self, vectorizer, model):
        logging.info("the vectorizer is: {}".format(vectorizer))
        
       # if self.vectorizer == 'tfidf':
       #     VECT = TfidfVectorizer()

      #  elif self.vectorizer == 'count':
      #      VECT = CountVectorizer
            
           # logging.info('... Loading W2V model....:{}'.format(self.model['filename']))
        vec = {}   
        vec['filename'] = vectorizer
        if vectorizer == 'w2v_count':
            s = embeddingvectorizer.EmbeddingCountVectorizer(model['gensimmodel'], 'mean')
        elif vectorizer == 'w2v_tfidf':
            s = embeddingvectorizer.EmbeddingTfidfVectorizer(model['gensimmodel'], 'mean')
        vec['vectorizer'] = s
        print("loaded vectorizer, named: {}".format(vec['filename']))
        yield vec


    def gridsearch_with_classifiers_embeddings(self):
        class_report = []
        results = []
        
        for model in self.get_w2v_model():
            for v in ["w2v_count", "w2v_tfidf"]:
                for vec in self.get_vectorizer(v, model):
                    print("loaded the vectorizer: {}".format(vec['filename'])) 
                    for name, classifier, params in zip(self.names, self.classifiers, self.parameters):
                        my_dict = {}
                        
                        logging.info("Starting gridsearch CV..")
                        logging.info("Classifier name: {}\n classifier:{}\n params{}\n".format(name, classifier, params)) 

                        clf_pipe = Pipeline([ ('vect', vec['vectorizer']), ('clf', classifier), ])

                        gs_clf = GridSearchCV(clf_pipe, param_grid=params, cv=2)
                        clf = gs_clf.fit(self.X_train, self.y_train)
                        score = clf.score(self.X_test, self.y_test)

                        logging.info("{} score: {}".format(name, score))
                        logging.info("{} are the best estimators".format(clf.best_estimator_))

                        results_to_dict = classification_report((clf.best_estimator_.predict(self.X_test)), self.y_test, output_dict= True)

                        results_to_dict['classifier'] = name
                        results_to_dict['parameters'] = clf.best_params_
                        results_to_dict['vectorizer'] = vec['filename']
                        results_to_dict['model'] = model['filename']

                        logging.info("Created dictionary with classification report: \n\n{}".format(results_to_dict))
                        class_report.append(results_to_dict)
                        
                        y_hats = clf.predict(self.X_test)
                        results.append({"predicted": y_hats,
                                        "actual" : self.y_test.values  ,
                                        "classifier": name} )
                        
        return class_report, results

    def gridsearch_with_classifiers_baseline(self):
        class_report = []
        results = []
        
        for vec, n in zip([CountVectorizer(), TfidfVectorizer()], ["Count", "TfidF"]):
            
            print("loaded the vectorizer: {}\n\n\{}".format(n, vec)) 
            
            for name, classifier, params in zip(self.names, self.classifiers, self.parameters):
                my_dict = {}

                logging.info("Starting gridsearch CV..")
                logging.info("Classifier name: {}\n classifier:{}\n params{}\n".format(name, classifier, params)) 

                clf_pipe = Pipeline([ ('vect', vec), ('clf', classifier), ])

                gs_clf = GridSearchCV(clf_pipe, param_grid=params, cv=2)
                clf = gs_clf.fit(self.X_train, self.y_train)
                score = clf.score(self.X_test, self.y_test)

                logging.info("{} score: {}".format(name, score))
                logging.info("{} are the best estimators".format(clf.best_estimator_))

                results_to_dict = classification_report((clf.best_estimator_.predict(self.X_test)), self.y_test, output_dict= True)

                results_to_dict['classifier'] = name
                results_to_dict['parameters'] = clf.best_params_
                results_to_dict['vectorizer'] = n
                results_to_dict['model'] = "baseline"
                
                logging.info("Created dictionary with classification report: \n\n{}".format(results_to_dict))
                class_report.append(results_to_dict)

                y_hats = clf.predict(self.X_test)
                results.append({"predicted": y_hats,
                                "actual" : self.y_test.values  ,
                                "classifier": name} )
                
        return class_report, results


def get_scores(path_to_data, path_to_embeddings, dataset, outputpath):
    a = classifier_analyzer(path_to_data=path_to_data, path_to_embeddings=path_to_embeddings, dataset=dataset)
    class_report, results = a.gridsearch_with_classifiers_baseline()

    fname_accuracy = '{}baseline_classreport.json'.format(outputpath)
    fname_true_predicted = '{}baseline_true_predicted.json'.format(outputpath)

    with open(fname_accuracy, mode = 'w') as fo:
        json.dump(class_report, fo)

    data = pd.DataFrame.from_dict(results)

    predicted = data.predicted.apply(pd.Series).merge(data, right_index = True, left_index = True) \
        .drop(["predicted"], axis = 1).melt(id_vars = ['classifier'], value_name = "Predicted label")

    actual = data.actual.apply(pd.Series).merge(data, right_index = True, left_index = True) \
        .drop(["predicted"], axis = 1).melt(id_vars = ['classifier'], value_name = "Actual label")

    df = pd.merge(predicted, actual, how = 'inner', left_index = True, right_index = True)
    
    df['Classifier'] = df['classifier_x']
    df = df[df.variable_x != 'actual']
    df = df[['Predicted label', 'Actual label', 'Classifier']]

    df.to_json(fname_true_predicted)

In [4]:
df = pd.read_pickle(path_to_data + dataset)
test_size = 0.2
data = df['text']
labels = df['topic']
# validation data
data, validation_set = train_test_split(data, labels, test_size = 0.1)
#self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.data, self.labels, test_size=self.test_size, random_state=42)

data

ValueError: too many values to unpack (expected 2)

In [ ]:
if __name__ == "__main__":

    logger = logging.getLogger()
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
    logging.root.setLevel(level=logging.INFO)
    
    get_scores(path_to_embeddings=path_to_embeddings,path_to_data=path_to_data, dataset=dataset, outputpath = outputpath)

#"w2v_count", "w2v_tfidf", "count", "tfidf"

print("DOOOOOOOOOOOOONEEE")